In [1]:
from modules.environment_manager import get_notebook_path, enforce_venv, get_persistent_config_data

NOTEBOOK_NAME = "inversion_result_analysis.ipynb"

NOTEBOOK_PATH = get_notebook_path(NOTEBOOK_NAME)
enforce_venv(NOTEBOOK_PATH)
THREADS, MEMORY, HSI_CONFIG, PROJECT_FOLDER, OUTPUT_FOLDER = get_persistent_config_data(NOTEBOOK_PATH)

In [2]:
from PIL import Image
import numpy as np
import pandas as pd

DATA_FOLDER = PROJECT_FOLDER / "data"
PROCESSED_FOLDER = DATA_FOLDER / "processed"

hsi_name_0 = input("Please Enter: hog_island_2019_0 hsi name in hsi_config.json")
hsi_name_1 = input("Please Enter: hog_island_2019_1 hsi name in hsi_config.json")

hsi_name_0 = hsi_name_0 if hsi_name_0 else "uas_hsi_small"
hsi_name_1 = hsi_name_1 if hsi_name_1 else "uas_hsi_large"

In [3]:
def get_inv_df(hsi_name):
    inv_path = OUTPUT_FOLDER / f"{hsi_name}_inv_res.npy"
    inv_res = np.load(inv_path)

    if inv_res.shape[0] == 3705:
        hog_island_id = 0
    elif inv_res.shape[0] == 5321:
        hog_island_id = 1
    else:
        raise RuntimeError("Inversion dimensions do not match any hog_island_2019 images.")

    success = inv_res[:, :, 0]
    n = inv_res[:, :, 1]
    cab = inv_res[:, :, 2]
    ccx = inv_res[:, :, 3]
    ewt = inv_res[:, :, 4]
    lma = inv_res[:, :, 5]
    lai = inv_res[:, :, 6]
    psoil = inv_res[:, :, 7]
    rsoil = inv_res[:, :, 8]
    mask = inv_res[:, :, 9]

    df = pd.DataFrame(
        columns=["vp_names", "img_idx", "pix_y", "pix_x", "n", "cab", "ccx", "ewt", "lma", "lai", "psoil", "rsoil"]
    )

    vp_lookup = np.array(
        Image.open(PROCESSED_FOLDER / f"vp_lookup_hog_island_2019_{hog_island_id}.png").convert("L"), dtype=np.uint8
    )

    df_idx = 0
    for i in range(vp_lookup.shape[0]):
        for j in range(vp_lookup.shape[1]):
            if success[i, j] > 0.99:
                vp_name = "vp_" + str(vp_lookup[i, j]).zfill(3)
                df.loc[df_idx] = [
                    vp_name,
                    hog_island_id,
                    i,
                    j,
                    n[i, j],
                    cab[i, j],
                    ccx[i, j],
                    ewt[i, j],
                    lma[i, j],
                    lai[i, j],
                    psoil[i, j],
                    rsoil[i, j],
                ]
                df_idx += 1
    return df


inv_df_0 = get_inv_df(hsi_name=hsi_name_0)
inv_df_1 = get_inv_df(hsi_name=hsi_name_1)
inv_res_df = pd.concat([inv_df_0, inv_df_1], ignore_index=True)
inv_res_df

,vp_names,img_idx,pix_y,pix_x,n,cab,ccx,ewt,lma,lai,psoil,rsoil
0,vp_030,0,1081,1510,1.095213,10.000000,8.267910,0.001941,0.034223,5.163403,0.719185,0.847475
1,vp_030,0,1082,1510,1.136846,10.000000,7.552181,0.001846,0.024383,4.081995,0.804409,0.737731
2,vp_030,0,1082,1511,1.140723,10.000000,7.461852,0.001905,0.032575,6.530915,0.919134,0.598911
3,vp_030,0,1082,1512,1.184393,10.000000,8.097132,0.001956,0.038317,5.216523,0.739252,0.733451
4,vp_030,0,1083,1509,1.000724,10.000000,7.000000,0.001890,0.027986,4.502490,0.866298,0.934582
...,...,...,...,...,...,...,...,...,...,...,...,...
3409,vp_080,1,2907,2839,1.033778,10.338265,7.000000,0.002000,0.076357,4.403463,0.704601,0.710270
3410,vp_080,1,2907,2840,1.121261,10.664826,7.515886,0.001982,0.048242,6.892568,0.709376,0.713260
3411,vp_080,1,2907,2841,1.028299,11.000689,7.000000,0.001978,0.069510,5.195986,0.741228,0.703044
3412,vp_080,1,2907,2842,1.028299,11.000689,7.000000,0.001978,0.069510,5.195986,0.741228,0.703044


In [4]:
print(inv_res_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   vp_names  3414 non-null   object 
 1   img_idx   3414 non-null   int64  
 2   pix_y     3414 non-null   int64  
 3   pix_x     3414 non-null   int64  
 4   n         3414 non-null   float64
 5   cab       3414 non-null   float64
 6   ccx       3414 non-null   float64
 7   ewt       3414 non-null   float64
 8   lma       3414 non-null   float64
 9   lai       3414 non-null   float64
 10  psoil     3414 non-null   float64
 11  rsoil     3414 non-null   float64
dtypes: float64(8), int64(3), object(1)
memory usage: 320.2+ KB
None


In [5]:
inv_res_df.to_csv(PROCESSED_FOLDER / "inv_res_hog_island_2019.csv")

In [6]:
inverted_vps = inv_res_df["vp_names"].unique()
inverted_vps.sort()
print(inverted_vps)
print("\nCount: ", len(inverted_vps))

['vp_003' 'vp_004' 'vp_005' 'vp_006' 'vp_008' 'vp_009' 'vp_010' 'vp_011'
 'vp_013' 'vp_014' 'vp_015' 'vp_016' 'vp_017' 'vp_018' 'vp_019' 'vp_020'
 'vp_021' 'vp_022' 'vp_023' 'vp_024' 'vp_025' 'vp_027' 'vp_028' 'vp_029'
 'vp_030' 'vp_037' 'vp_038' 'vp_039' 'vp_040' 'vp_046' 'vp_047' 'vp_050'
 'vp_051' 'vp_054' 'vp_062' 'vp_065' 'vp_071' 'vp_072' 'vp_073' 'vp_074'
 'vp_078' 'vp_080' 'vp_082' 'vp_085' 'vp_091']

Count:  45


In [7]:
print(inv_res_df.describe())

           img_idx        pix_y        pix_x            n          cab  \
count  3414.000000  3414.000000  3414.000000  3414.000000  3414.000000   
mean      0.316344  1875.260691  2053.992970     1.121723    12.743518   
std       0.465117   512.024204   544.463564     0.079230    10.812077   
min       0.000000  1081.000000   919.000000     1.000000    10.000000   
25%       0.000000  1409.000000  1795.000000     1.052231    10.000000   
50%       0.000000  1856.000000  1938.000000     1.115431    10.000000   
75%       1.000000  2242.750000  2343.000000     1.195103    10.537966   
max       1.000000  2908.000000  3222.000000     1.250000   101.000000   

               ccx          ewt          lma          lai        psoil  \
count  3414.000000  3414.000000  3414.000000  3414.000000  3414.000000   
mean      8.190639     0.001905     0.056832     5.216293     0.684194   
std       0.644351     0.000111     0.025622     1.300515     0.137910   
min       7.000000     0.001500     0

In [8]:
vp_master_df = pd.read_excel(DATA_FOLDER / "VP_MASTER_DATASET_2019_VALUES_ONLY.xlsx")
print(vp_master_df.head())

     VP        SITE  Northing_avg_m  Easting_avg_m  Elevation_avg_adj_m  \
0  VP01  BROADWATER    4.138686e+06   437052.00950             -0.10400   
1  VP02  BROADWATER    4.138681e+06   437048.70275             -0.09775   
2  VP03  BROADWATER    4.138681e+06   437052.33900             -0.09200   
3  VP04  BROADWATER    4.138678e+06   437048.65600             -0.06125   
4  VP05  BROADWATER    4.138679e+06   437059.82025              0.00025   

   Elevation_avg_m  BD_VP_A_7_2019_g_cm-3  BD_VP_B_7_2019_g_cm-3  \
0          1.89600               0.455712               0.544917   
1          1.90225               0.502429               0.480717   
2          1.90800               0.527729               0.493317   
3          1.93875               0.458996               0.538683   
4          2.00025               0.406162               0.394317   

   BD_VP_AVG_7_2019_g_cm-3  SOM_VP_A_7_2019_PERCENT  ...  \
0                 0.500314                 8.898159  ...   
1                 0.

In [9]:
# Extract the numeric part of vp_names and create a new column for merging
inv_res_df["vp_names"] = inv_res_df["vp_names"].str.extract(r"(\d+)").astype(int)

# Extract the numeric part of VP from the VP master dataset for consistency
vp_master_df["VP"] = vp_master_df["VP"].str.extract(r"(\d+)").astype(int)

# Merge the datasets on the numeric VP column
merged_df = pd.merge(inv_res_df, vp_master_df, left_on="vp_names", right_on="VP")
merged_df.rename(columns={"vp_names":"vp_num"}, inplace=True)
merged_df.drop(columns=["VP"], inplace=True)

print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   vp_num                            3414 non-null   int32  
 1   img_idx                           3414 non-null   int64  
 2   pix_y                             3414 non-null   int64  
 3   pix_x                             3414 non-null   int64  
 4   n                                 3414 non-null   float64
 5   cab                               3414 non-null   float64
 6   ccx                               3414 non-null   float64
 7   ewt                               3414 non-null   float64
 8   lma                               3414 non-null   float64
 9   lai                               3414 non-null   float64
 10  psoil                             3414 non-null   float64
 11  rsoil                             3414 non-null   float64
 12  SITE  

In [10]:
merged_df_object_cols = merged_df.select_dtypes(include=["object"])
merged_df_object_cols.head()

,SITE,SWC_VP_A_7_2019_g_g_soil_-1,SWC_VP_B_7_2019_g_g_soil_-1,SWC_VP_AVG_7_2019_g_g_soil-1,S_alt_Biomass_10_2019_g_m-2,Salicornia_biomass_10_2019_g_m-2,TOTAL_AGB_10_2019_g_m-2,CHL-A_VP_AVG_10_2019_mg_m-2,PHP_VP_AVG_10_2019_mg_m-2
0,BROADWATER,0.915171,0.794866,0.855018,ND,ND,ND,ND,ND
1,BROADWATER,0.915171,0.794866,0.855018,ND,ND,ND,ND,ND
2,BROADWATER,0.915171,0.794866,0.855018,ND,ND,ND,ND,ND
3,BROADWATER,0.915171,0.794866,0.855018,ND,ND,ND,ND,ND
4,BROADWATER,0.915171,0.794866,0.855018,ND,ND,ND,ND,ND


In [11]:
merged_df_object_cols.nunique()

SITE                                 2
SWC_VP_A_7_2019_g_g_soil_-1         45
SWC_VP_B_7_2019_g_g_soil_-1         45
SWC_VP_AVG_7_2019_g_g_soil-1        45
S_alt_Biomass_10_2019_g_m-2         15
Salicornia_biomass_10_2019_g_m-2     2
TOTAL_AGB_10_2019_g_m-2             15
CHL-A_VP_AVG_10_2019_mg_m-2         14
PHP_VP_AVG_10_2019_mg_m-2           15
dtype: int64

In [12]:
merged_df_object_cols.replace("ND", np.nan, inplace=True)
merged_df_object_cols.head()

C:\Users\holde\AppData\Local\Temp\ipykernel_19020\2926888994.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df_object_cols.replace("ND", np.nan, inplace=True)


,SITE,SWC_VP_A_7_2019_g_g_soil_-1,SWC_VP_B_7_2019_g_g_soil_-1,SWC_VP_AVG_7_2019_g_g_soil-1,S_alt_Biomass_10_2019_g_m-2,Salicornia_biomass_10_2019_g_m-2,TOTAL_AGB_10_2019_g_m-2,CHL-A_VP_AVG_10_2019_mg_m-2,PHP_VP_AVG_10_2019_mg_m-2
0,BROADWATER,0.915171,0.794866,0.855018,NaN,NaN,NaN,NaN,NaN
1,BROADWATER,0.915171,0.794866,0.855018,NaN,NaN,NaN,NaN,NaN
2,BROADWATER,0.915171,0.794866,0.855018,NaN,NaN,NaN,NaN,NaN
3,BROADWATER,0.915171,0.794866,0.855018,NaN,NaN,NaN,NaN,NaN
4,BROADWATER,0.915171,0.794866,0.855018,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_df_object_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   SITE                              3414 non-null   object 
 1   SWC_VP_A_7_2019_g_g_soil_-1       3414 non-null   float64
 2   SWC_VP_B_7_2019_g_g_soil_-1       3414 non-null   float64
 3   SWC_VP_AVG_7_2019_g_g_soil-1      3414 non-null   float64
 4   S_alt_Biomass_10_2019_g_m-2       1080 non-null   float64
 5   Salicornia_biomass_10_2019_g_m-2  1080 non-null   float64
 6   TOTAL_AGB_10_2019_g_m-2           1080 non-null   float64
 7   CHL-A_VP_AVG_10_2019_mg_m-2       1080 non-null   float64
 8   PHP_VP_AVG_10_2019_mg_m-2         1080 non-null   float64
dtypes: float64(8), object(1)
memory usage: 240.2+ KB


In [14]:
merged_df_object_cols['SITE'] = pd.Categorical(merged_df_object_cols['SITE'])
merged_df_object_cols['SITE_ID'] = merged_df_object_cols['SITE'].cat.codes
site_from_id_lookup = dict(zip(merged_df_object_cols['SITE'].cat.codes, merged_df_object_cols['SITE']))

merged_df_object_cols.drop(columns=["SITE"], inplace=True)
merged_df_object_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   SWC_VP_A_7_2019_g_g_soil_-1       3414 non-null   float64
 1   SWC_VP_B_7_2019_g_g_soil_-1       3414 non-null   float64
 2   SWC_VP_AVG_7_2019_g_g_soil-1      3414 non-null   float64
 3   S_alt_Biomass_10_2019_g_m-2       1080 non-null   float64
 4   Salicornia_biomass_10_2019_g_m-2  1080 non-null   float64
 5   TOTAL_AGB_10_2019_g_m-2           1080 non-null   float64
 6   CHL-A_VP_AVG_10_2019_mg_m-2       1080 non-null   float64
 7   PHP_VP_AVG_10_2019_mg_m-2         1080 non-null   float64
 8   SITE_ID                           3414 non-null   int8   
dtypes: float64(8), int8(1)
memory usage: 216.8 KB


In [15]:
shared_columns = merged_df.columns.intersection(merged_df_object_cols.columns)
new_columns = merged_df_object_cols.columns.difference(merged_df.columns)
merged_df[shared_columns] = merged_df_object_cols[shared_columns]
merged_df = pd.concat([merged_df, merged_df_object_cols[new_columns]], axis=1)
merged_df.drop(columns=["SITE"], inplace=True)
merged_df.to_csv(PROCESSED_FOLDER / "inv_res_master_merged_hog_island_2019.csv")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   vp_num                            3414 non-null   int32  
 1   img_idx                           3414 non-null   int64  
 2   pix_y                             3414 non-null   int64  
 3   pix_x                             3414 non-null   int64  
 4   n                                 3414 non-null   float64
 5   cab                               3414 non-null   float64
 6   ccx                               3414 non-null   float64
 7   ewt                               3414 non-null   float64
 8   lma                               3414 non-null   float64
 9   lai                               3414 non-null   float64
 10  psoil                             3414 non-null   float64
 11  rsoil                             3414 non-null   float64
 12  Northi

In [16]:
inv_res_master_merged_stats_hog_island_2019 = merged_df.describe()
inv_res_master_merged_stats_hog_island_2019.to_csv(PROCESSED_FOLDER / "inv_res_master_merged_stats_hog_island_2019.csv")
inv_res_master_merged_stats_hog_island_2019

,vp_num,img_idx,pix_y,pix_x,n,cab,ccx,ewt,lma,lai,...,oyster_total_7_2019,oyster_percent_cover_7_2019,geukensia_total_7_2019,geukensia_percent_cover_7_2019,S_alt_Biomass_10_2019_g_m-2,Salicornia_biomass_10_2019_g_m-2,TOTAL_AGB_10_2019_g_m-2,CHL-A_VP_AVG_10_2019_mg_m-2,PHP_VP_AVG_10_2019_mg_m-2,SITE_ID
count,3414.000000,3414.000000,3414.000000,3414.000000,3414.000000,3414.000000,3414.000000,3414.000000,3414.000000,3414.000000,...,3414.000000,3414.000000,3414.000000,3414.000000,1080.000000,1080.0,1080.000000,1080.000000,1080.000000,3414.000000
mean,36.475688,0.316344,1875.260691,2053.992970,1.121723,12.743518,8.190639,0.001905,0.056832,5.216293,...,1.649092,1.122144,2.328354,1.317516,498.466527,0.0,498.466527,116.841549,87.968751,0.316344
std,25.962794,0.465117,512.024204,544.463564,0.079230,10.812077,0.644351,0.000111,0.025622,1.300515,...,3.188005,2.176780,6.282744,3.319663,434.643545,0.0,434.643545,34.027655,48.108996,0.465117
min,3.000000,0.000000,1081.000000,919.000000,1.000000,10.000000,7.000000,0.001500,0.002057,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,55.313505,27.805332,0.000000
25%,15.000000,0.000000,1409.000000,1795.000000,1.052231,10.000000,7.691352,0.001874,0.037441,4.776210,...,0.000000,0.000000,0.000000,0.000000,231.223126,0.0,231.223126,92.637726,60.478831,0.000000
50%,29.000000,0.000000,1856.000000,1938.000000,1.115431,10.000000,8.386750,0.001945,0.050809,5.454352,...,0.000000,0.000000,0.000000,0.000000,275.102190,0.0,275.102190,127.695581,70.133039,0.000000
75%,54.000000,1.000000,2242.750000,2343.000000,1.195103,10.537966,8.725847,0.001984,0.076461,5.974919,...,2.000000,1.000000,1.000000,1.000000,780.274278,0.0,780.274278,147.030520,95.690238,1.000000
max,91.000000,1.000000,2908.000000,3222.000000,1.250000,101.000000,9.000000,0.002000,0.100000,9.815744,...,16.000000,10.000000,33.000000,15.000000,1577.424571,0.0,1577.424571,165.373922,208.004515,1.000000


In [17]:
inv_corr_df = merged_df.corr()
inv_corr_df = inv_corr_df.dropna(axis=0, how="all").dropna(axis=1, how="all")
inv_corr_df

,vp_num,img_idx,pix_y,pix_x,n,cab,ccx,ewt,lma,lai,...,crab-fiddler_total_7_2019,oyster_total_7_2019,oyster_percent_cover_7_2019,geukensia_total_7_2019,geukensia_percent_cover_7_2019,S_alt_Biomass_10_2019_g_m-2,TOTAL_AGB_10_2019_g_m-2,CHL-A_VP_AVG_10_2019_mg_m-2,PHP_VP_AVG_10_2019_mg_m-2,SITE_ID
vp_num,1.000000,0.874140,0.676205,0.643174,-0.056781,0.196000,0.028184,0.092425,-0.000157,-0.055999,...,0.170837,0.020368,0.012130,0.121558,0.002986,-0.092899,-0.092899,-0.431075,0.099855,0.874140
img_idx,0.874140,1.000000,0.780608,0.818977,-0.017534,0.223716,0.027268,0.102816,-0.115401,-0.025409,...,0.349786,-0.070744,-0.091713,0.149133,0.063396,NaN,NaN,NaN,NaN,1.000000
pix_y,0.676205,0.780608,1.000000,0.659138,0.026879,0.083054,0.060281,0.152353,-0.159764,0.031090,...,0.258860,-0.087793,-0.044295,-0.023230,-0.034403,-0.767080,-0.767080,-0.459459,-0.323918,0.780608
pix_x,0.643174,0.818977,0.659138,1.000000,-0.089065,0.121430,-0.067989,0.005348,0.065245,-0.028371,...,0.252024,-0.255776,-0.304905,0.121219,0.081771,-0.380754,-0.380754,-0.210492,-0.224133,0.818977
n,-0.056781,-0.017534,0.026879,-0.089065,1.000000,-0.075312,0.190436,0.168481,-0.371339,0.056491,...,0.057925,-0.016532,-0.023545,-0.049504,-0.033002,-0.001401,-0.001401,0.073401,-0.034161,-0.017534
cab,0.196000,0.223716,0.083054,0.121430,-0.075312,1.000000,-0.037095,-0.137580,0.050173,-0.442991,...,0.047387,0.096594,0.100031,0.327051,0.275929,0.475077,0.475077,-0.049562,0.423228,0.223716
ccx,0.028184,0.027268,0.060281,-0.067989,0.190436,-0.037095,1.000000,0.292676,-0.266899,0.061404,...,0.016994,-0.021962,-0.022392,-0.041527,-0.050022,-0.019142,-0.019142,0.129843,-0.043492,0.027268
ewt,0.092425,0.102816,0.152353,0.005348,0.168481,-0.137580,0.292676,1.000000,-0.377482,0.149375,...,0.086360,0.019477,0.032649,0.014093,0.019838,-0.072027,-0.072027,0.001032,-0.071871,0.102816
lma,-0.000157,-0.115401,-0.159764,0.065245,-0.371339,0.050173,-0.266899,-0.377482,1.000000,-0.195884,...,-0.304789,-0.125900,-0.146673,-0.161609,-0.205255,-0.253460,-0.253460,-0.234492,-0.055592,-0.115401
lai,-0.055999,-0.025409,0.031090,-0.028371,0.056491,-0.442991,0.061404,0.149375,-0.195884,1.000000,...,0.062361,0.022534,0.021305,-0.109646,-0.083156,-0.226137,-0.226137,0.018977,-0.214810,-0.025409


In [18]:
inv_corr_df.to_csv(PROCESSED_FOLDER / "inv_res_corr_all_hog_island_2019.csv")
inv_corr_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, vp_num to SITE_ID
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   vp_num                            44 non-null     float64
 1   img_idx                           40 non-null     float64
 2   pix_y                             44 non-null     float64
 3   pix_x                             44 non-null     float64
 4   n                                 44 non-null     float64
 5   cab                               44 non-null     float64
 6   ccx                               44 non-null     float64
 7   ewt                               44 non-null     float64
 8   lma                               44 non-null     float64
 9   lai                               44 non-null     float64
 10  psoil                             44 non-null     float64
 11  rsoil                             44 non-null     float64
 12  North

In [19]:
inv_corr_df_styled = inv_corr_df.style.background_gradient(
    cmap='plasma_r',
    vmin=inv_corr_df.min().min(),
    vmax=inv_corr_df.max().max(),
)
inv_corr_df_styled 

,vp_num,img_idx,pix_y,pix_x,n,cab,ccx,ewt,lma,lai,psoil,rsoil,Northing_avg_m,Easting_avg_m,Elevation_avg_adj_m,Elevation_avg_m,BD_VP_A_7_2019_g_cm-3,BD_VP_B_7_2019_g_cm-3,BD_VP_AVG_7_2019_g_cm-3,SOM_VP_A_7_2019_PERCENT,SWC_VP_A_7_2019_g_g_soil_-1,SWC_VP_B_7_2019_g_g_soil_-1,SWC_VP_AVG_7_2019_g_g_soil-1,CHL-A_VP_AVG_7_2019_mg_m-2,PHP_VP_AVG_7_2019_mg_m-1,SALINITY_ppt_7_2019,10_cm_Bulk_Density_7_2019_g_cm-2,10_cm_SOM_7_2019_percent,S_alt_Biomass_7_2019_g_m-2,Salicornia_biomass_7_2019_g_m-2,TOTAL_AGB_7_2019_g_m-2,ilyanassa_total_7_2019,littoraria_total_7_2019,crab-hooded_total_7_2019,crab-fiddler_total_7_2019,oyster_total_7_2019,oyster_percent_cover_7_2019,geukensia_total_7_2019,geukensia_percent_cover_7_2019,S_alt_Biomass_10_2019_g_m-2,TOTAL_AGB_10_2019_g_m-2,CHL-A_VP_AVG_10_2019_mg_m-2,PHP_VP_AVG_10_2019_mg_m-2,SITE_ID
vp_num,1.000000,0.874140,0.676205,0.643174,-0.056781,0.196000,0.028184,0.092425,-0.000157,-0.055999,0.016652,0.058008,-0.871777,-0.882436,-0.262311,-0.262311,0.767477,0.762781,0.774434,-0.731445,-0.742387,-0.759775,-0.756980,-0.177522,-0.233432,0.326153,0.820172,-0.811367,-0.107016,0.156646,-0.106617,-0.222844,0.046420,-0.230579,0.170837,0.020368,0.012130,0.121558,0.002986,-0.092899,-0.092899,-0.431075,0.099855,0.874140
img_idx,0.874140,1.000000,0.780608,0.818977,-0.017534,0.223716,0.027268,0.102816,-0.115401,-0.025409,0.069223,0.085321,-0.997928,-0.988475,-0.256862,-0.256862,0.859027,0.815783,0.847633,-0.834964,-0.828186,-0.812156,-0.826628,0.003594,-0.166800,0.393010,0.921003,-0.928805,-0.086143,0.209570,-0.085601,-0.390072,0.067251,-0.189525,0.349786,-0.070744,-0.091713,0.149133,0.063396,nan,nan,nan,nan,1.000000
pix_y,0.676205,0.780608,1.000000,0.659138,0.026879,0.083054,0.060281,0.152353,-0.159764,0.031090,0.145216,0.113669,-0.819195,-0.766551,-0.193197,-0.193197,0.670551,0.594749,0.640426,-0.580772,-0.599221,-0.608103,-0.608405,-0.008168,-0.246950,0.275209,0.635762,-0.646565,-0.109976,0.040095,-0.109886,-0.320963,0.093316,-0.173012,0.258860,-0.087793,-0.044295,-0.023230,-0.034403,-0.767080,-0.767080,-0.459459,-0.323918,0.780608
pix_x,0.643174,0.818977,0.659138,1.000000,-0.089065,0.121430,-0.067989,0.005348,0.065245,-0.028371,-0.048272,-0.006316,-0.819849,-0.722681,-0.143463,-0.143463,0.612826,0.525281,0.576070,-0.588167,-0.571778,-0.521728,-0.551075,0.068028,-0.147334,0.477510,0.686534,-0.704958,-0.099211,0.167218,-0.098783,-0.366761,0.360034,-0.103860,0.252024,-0.255776,-0.304905,0.121219,0.081771,-0.380754,-0.380754,-0.210492,-0.224133,0.818977
n,-0.056781,-0.017534,0.026879,-0.089065,1.000000,-0.075312,0.190436,0.168481,-0.371339,0.056491,0.105884,0.057411,0.013427,-0.002677,0.141277,0.141277,0.050347,0.089802,0.070881,-0.067004,-0.045685,-0.072104,-0.059346,-0.025485,-0.010475,-0.218869,0.042865,-0.032331,0.019074,-0.087921,0.018844,-0.006162,-0.091790,0.039329,0.057925,-0.016532,-0.023545,-0.049504,-0.033002,-0.001401,-0.001401,0.073401,-0.034161,-0.017534
cab,0.196000,0.223716,0.083054,0.121430,-0.075312,1.000000,-0.037095,-0.137580,0.050173,-0.442991,-0.215877,-0.004356,-0.213583,-0.237539,-0.188395,-0.188395,0.116869,0.127873,0.123846,-0.156026,-0.172223,-0.177616,-0.176293,0.028466,0.284565,0.080397,0.227892,-0.193032,0.163196,0.085447,0.163447,-0.098071,0.019270,-0.017152,0.047387,0.096594,0.100031,0.327051,0.275929,0.475077,0.475077,-0.049562,0.423228,0.223716
ccx,0.028184,0.027268,0.060281,-0.067989,0.190436,-0.037095,1.000000,0.292676,-0.266899,0.061404,0.116037,0.068348,-0.031101,-0.050774,0.104481,0.104481,0.082436,0.096597,0.090588,-0.076743,-0.082388,-0.084591,-0.084145,-0.041294,-0.047507,-0.094136,0.082054,-0.066674,-0.036648,-0.047473,-0.036780,0.074069,-0.068610,-0.060461,0.016994,-0.021962,-0.022392,-0.041527,-0.050022,-0.019142,-0.019142,0.129843,-0.043492,0.027268
ewt,0.092425,0.102816,0.152353,0.005348,0.168481,-0.137580,0.292676,1.000000,-0.377482,0.149375,0.313085,0.184013,-0.109879,-0.122376,0.114888,0.114888,0.152429,0.157232,0.156707,-0.120256,-0.160542,-0.174754,-0.168960,-0.04

In [20]:
inv_corr_df_focused = inv_corr_df.drop(columns=["vp_num","img_idx","pix_y","pix_x"]).drop(["vp_num","img_idx","pix_y","pix_x", "n", "cab", "ccx", "ewt", "lma", "lai", "psoil", "rsoil"])
inv_corr_df_focused = inv_corr_df_focused.drop(columns=inv_corr_df_focused.columns[8:])
inv_corr_df_focused.to_csv(PROCESSED_FOLDER / "inv_res_corr_hog_island_2019.csv")
inv_corr_df_focused_styled = inv_corr_df_focused.style.background_gradient(
    cmap='plasma_r',
    vmin=inv_corr_df_focused.min().min(),
    vmax=inv_corr_df_focused.max().max(),
)
inv_corr_df_focused_styled

,n,cab,ccx,ewt,lma,lai,psoil,rsoil
Northing_avg_m,0.013427,-0.213583,-0.031101,-0.109879,0.121830,0.020088,-0.078213,-0.089685
Easting_avg_m,-0.002677,-0.237539,-0.050774,-0.122376,0.156738,0.023077,-0.096284,-0.104494
Elevation_avg_adj_m,0.141277,-0.188395,0.104481,0.114888,-0.247265,0.116677,0.190453,0.141083
Elevation_avg_m,0.141277,-0.188395,0.104481,0.114888,-0.247265,0.116677,0.190453,0.141083
BD_VP_A_7_2019_g_cm-3,0.050347,0.116869,0.082436,0.152429,-0.209483,0.036132,0.161174,0.132792
BD_VP_B_7_2019_g_cm-3,0.089802,0.127873,0.096597,0.157232,-0.271484,0.036170,0.178687,0.152143
BD_VP_AVG_7_2019_g_cm-3,0.070881,0.123846,0.090588,0.156707,-0.243336,0.036591,0.171976,0.144177
SOM_VP_A_7_2019_PERCENT,-0.067004,-0.156026,-0.076743,-0.120256,0.220240,-0.021366,-0.119930,-0.103644
SWC_VP_A_7_2019_g_g_soil_-1,-0.045685,-0.172223,-0.082388,-0.160542,0.258287,-0.028145,-0.158938,-0.149360
SWC_VP_B_7_2019_g_g_soil_-1,-0.072104,-0.177616,-0.084591,-0.174754,0.291457,-0.042922,-0.190523,-0.178242
